In [1]:
import threading
import random
import time


numbers = []

filled_event = threading.Event()

def fill_list(size):
    global numbers
    for _ in range(size):
        numbers.append(random.randint(1, 100))
        time.sleep(0.01)
    filled_event.set()

def compute_sum():

    filled_event.wait()
    total = sum(numbers)
    print(f"Сумма элементов списка: {total}")

def compute_average():

    filled_event.wait()
    average = sum(numbers) / len(numbers)
    print(f"Среднеарифметическое значение: {average}")

def main():
    size = 20
    filler_thread = threading.Thread(target=fill_list, args=(size,))
    sum_thread = threading.Thread(target=compute_sum)
    avg_thread = threading.Thread(target=compute_average)

    filler_thread.start()

    sum_thread.start()
    avg_thread.start()


    filler_thread.join()
    sum_thread.join()
    avg_thread.join()

if __name__ == "__main__":
    main()

Сумма элементов списка: 907Среднеарифметическое значение: 45.35



In [ ]:
import threading
import random
import os
import math
import sys

stats = {
    'numbers_generated': 0,
    'primes_found': 0,
    'factorials_computed': 0
}

file_ready_event = threading.Event()

def is_prime(n):
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

def fill_file_with_random_numbers(file_path, count=100):
    with open(file_path, 'w') as f:
        for _ in range(count):
            num = random.randint(1, 10000)
            f.write(str(num) + '\n')
            stats['numbers_generated'] += 1

    file_ready_event.set()

def find_primes_in_file(input_file, output_file):

    file_ready_event.wait()
    primes = []
    with open(input_file, 'r') as f:
        for line in f:
            num = int(line.strip())
            if is_prime(num):
                primes.append(str(num))
                stats['primes_found'] += 1
    with open(output_file, 'w') as f:
        f.write('\n'.join(primes))
    print(f"Обнаружено простых чисел: {stats['primes_found']}")

def compute_factorials_in_file(input_file, output_file):

    file_ready_event.wait()
    with open(input_file, 'r') as f_in, open(output_file, 'w') as f_out:
        for line in f_in:
            num = int(line.strip())
            factorial = math.factorial(num)
            f_out.write(str(factorial) + '\n')
            stats['factorials_computed'] += 1
    print(f"Вычислено факториалов: {stats['factorials_computed']}")

def main():

    file_path = input("Введите путь к файлу: ")

    filler_thread = threading.Thread(target=fill_file_with_random_numbers, args=(file_path,))
    primes_thread = threading.Thread(target=find_primes_in_file, args=(file_path, 'primes.txt'))
    factorial_thread = threading.Thread(target=compute_factorials_in_file, args=(file_path, 'factorials.txt'))

    filler_thread.start()
    primes_thread.start()
    factorial_thread.start()

    filler_thread.join()
    primes_thread.join()
    factorial_thread.join()

    print("Статистика выполнения:")
    print(f"Всего сгенерировано чисел: {stats['numbers_generated']}")
    print(f"Обнаружено простых чисел: {stats['primes_found']}")
    print(f"Вычислено факториалов: {stats['factorials_computed']}")

if __name__ == "__main__":
    main()

Exception in thread Thread-6 (fill_file_with_random_numbers):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\User\AppData\Local\Temp\ipykernel_8652\3019674116.py", line 30, in fill_file_with_random_numbers
  File "C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py", line 324, in _modified_o

In [ ]:
import os
import shutil
import threading

class CopyStats:
    def __init__(self):
        self.files_copied = 0
        self.directories_created = 0

def copy_directory(src, dst, stats):
    for root, dirs, files in os.walk(src):

        rel_path = os.path.relpath(root, src)
        new_dir = os.path.join(dst, rel_path)
        os.makedirs(new_dir, exist_ok=True)
        stats.directories_created += 1
        print(f"Создана директория: {new_dir}")

        for file in files:
            src_file = os.path.join(root, file)
            dst_file = os.path.join(new_dir, file)
            shutil.copy2(src_file, dst_file)
            stats.files_copied += 1
            print(f"Скопирован файл: {src_file} -> {dst_file}")

def main():
    src_path = input("Введите путь к существующей директории: ")
    new_path = input("Введите путь к новой директории: ")

    if not os.path.isdir(src_path):
        print("Указанная исходная директория не существует.")
        return

    if os.path.exists(new_path):
        print("Целевая директория уже существует. Пожалуйста, выберите другой путь.")
        return

    stats = CopyStats()

    copy_thread = threading.Thread(target=copy_directory, args=(src_path, new_path, stats))
    copy_thread.start()
    copy_thread.join()

    print("\nКопирование завершено.")
    print(f"Создано директорий: {stats.directories_created}")
    print(f"Скопировано файлов: {stats.files_copied}")

if __name__ == "__main__":
    main()

KeyboardInterrupt: Interrupted by user

In [ ]:
import os
import threading
import re

files_found = []
files_with_word = 0
total_files_checked = 0
words_removed = 0

def load_forbidden_words(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return [line.strip() for line in f.readlines()]

def search_files(search_dir, search_word, output_filename):
    global files_found, files_with_word, total_files_checked
    for root, dirs, files in os.walk(search_dir):
        for filename in files:
            total_files_checked += 1
            file_path = os.path.join(root, filename)
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    content = f.read()
                    if search_word in content:
                        files_found.append(file_path)
                        files_with_word += 1
                        with open(output_filename, 'a', encoding='utf-8') as out_f:
                            out_f.write(f"\n{'='*20} {file_path} {'='*20}\n")
                            out_f.write(content + '\n')
            except:
                continue

def remove_forbidden_words(input_filename, forbidden_words):
    global words_removed
    pattern = re.compile('|'.join([re.escape(word) for word in forbidden_words]), re.IGNORECASE)
    try:
        with open(input_filename, 'r+', encoding='utf-8') as f:
            content = f.read()
            matches = pattern.findall(content)
            words_removed += len(matches)
            clean_content = pattern.sub('', content)
            f.seek(0)
            f.truncate()
            f.write(clean_content)
    except:
        pass

def main():
    search_dir = input("Введите путь к директории: ")
    search_word = input("Введите слово для поиска: ")
    forbidden_words_file = input("Введите путь к файлу с запрещенными словами: ")
    output_file = "merged.txt"

    forbidden_words = load_forbidden_words(forbidden_words_file)

    thread_search = threading.Thread(target=search_files, args=(search_dir, search_word, output_file))

    def second_thread():
        thread_search.join()
        remove_forbidden_words(output_file, forbidden_words)

    thread_remove = threading.Thread(target=second_thread)

    thread_search.start()
    thread_remove.start()

    thread_search.join()
    thread_remove.join()

    print(f"Всего проверено файлов: {total_files_checked}")
    print(f"Найдено файлов с искомым словом: {files_with_word}")
    print(f"Общий удаленных запрещенных слов: {words_removed}")

if __name__ == "__main__":
    main()